## An agent to validate radiology report based on ACR guidelines
The American College of Radiology (ACR), as the leading professional organization for radiologists, has established comprehensive guidelines for standardized radiology reporting to ensure diagnostic accuracy and patient care quality. In many healthcare settings, the current workflow requires senior radiologists to review and validate reports generated by junior colleagues—a critical but time-intensive process that creates bottlenecks in medical imaging departments already struggling with high workloads. To streamline this process while maintaining quality standards, Amazon NOVA's advanced multimodal AI capabilities can be leveraged to automatically evaluate radiology reports against ACR's established criteria, providing real-time feedback and guidance. This AI-assisted solution can instantly identify missing elements, suggest improvements based on ACR guidelines, and help junior radiologists generate more accurate and complete reports, ultimately reducing the review burden on senior radiologists while maintaining high reporting standards.

### Prepare the data
The ACR validation documents are in the folder ACRdocs. Upload the same in an S3 bucket. Change the name of the S3 bucket in the lambda function

In [1]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/typeguard-4.4.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/monai_deploy_app_sdk-0+untagged.502.gd03c0eb-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/pys3sync-0.1.2-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

### Configure `AWS_PROFILE`
Configure the `AWS_PROFILE` using the usual process as described [here](https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-files.html)

In [2]:
import os
os.environ['AWS_PROFILE'] = 'radiology-agent'

In [3]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.37.33


#### Configure the `boto3` clients

In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '575108919340')

### Write the agent instruction

In [8]:
# configuration variables
suffix = f"{region}-{account_id}"
stack_name = "RadiologyReportValidator"
agent_name = f"{stack_name}-Agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}-{stack_name}"
agent_role_name = f'AmazonBRkExRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for Validating Radiology Reports"
agent_instruction = "You are a Radiology Report Validator, helping junior radiologist \
                    write reports in adherence to the ACR guidance criterion. Does the radiology report adheres to the ACR guidelines mentioned in the document? \
                    Is it detailed enough to provide a diagnosis? \
                    Is the report missing any key anatomical structures? \
                    Does the report meet the \
                    quality standards of the ACR guidelines? Please provide a terse actionable feedback and do not try to summarize the report itself. ?"
agent_action_group_name = "RadiologyActionGroup"
agent_action_group_description = "Actions for Validating Radiology Reports or from Machine Learning Models"
agent_alias_name = f"{agent_name}-alias"


### Assign the roles

In [9]:
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
print(lambda_function_name)
lambda_function_arn = lambda_client.get_function(FunctionName=lambda_function_name)['Configuration']['FunctionArn']
# print(lambda_client.get_function(FunctionName=lambda_function_name))
lambda_client.get_function(FunctionName=lambda_function_name)["Configuration"]["FunctionArn"]

# attach the s3 full access policy to the Lambda function role
s3_full_access_policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
bedrock_policy_arn = "arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
AWSLambdaBasicExecutionRole_arn = "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"

# create role for the Lambda function
try:
    iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "lambda.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }),
        Description="Role for Lambda function to access S3 and other resources"
    )

    iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=s3_full_access_policy_arn
        
    )

except Exception as e:
    print("Error creating role: ", e)
    print("Role already exists. Continuing...")


RadiologyReportValidator-575108919340-Lambda
Error creating role:  An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name RadiologyReportValidator-LambdaExecutionRole already exists.
Role already exists. Continuing...


In [10]:
iam_client.attach_role_policy(
        RoleName=lambda_function_role,
        PolicyArn=AWSLambdaBasicExecutionRole_arn
    )
iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn=bedrock_policy_arn
)
# assign the created role to a lambda function
lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Role=iam_client.get_role(RoleName=lambda_function_role)['Role']['Arn'],
    Timeout=900
)

{'ResponseMetadata': {'RequestId': '180af65c-89ed-4efe-99d7-a9f9584ac7db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 12 Apr 2025 06:33:56 GMT',
   'content-type': 'application/json',
   'content-length': '1457',
   'connection': 'keep-alive',
   'x-amzn-requestid': '180af65c-89ed-4efe-99d7-a9f9584ac7db'},
  'RetryAttempts': 0},
 'FunctionName': 'RadiologyReportValidator-575108919340-Lambda',
 'FunctionArn': 'arn:aws:lambda:us-west-2:575108919340:function:RadiologyReportValidator-575108919340-Lambda',
 'Runtime': 'python3.13',
 'Role': 'arn:aws:iam::575108919340:role/RadiologyReportValidator-LambdaExecutionRole',
 'Handler': 'lambda_function.lambda_handler',
 'CodeSize': 6313,
 'Description': '',
 'Timeout': 900,
 'MemorySize': 128,
 'LastModified': '2025-04-12T06:33:56.000+0000',
 'CodeSha256': 'wsZt5cHtMZPJE9pF/xqhm4ll/6q/wpLaMQqn59Eh8VA=',
 'Version': '$LATEST',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': 'f659cb38-f940-48ab-a20e-62638c32f611',
 'State': 

#### Write and Upload the lambda function

In [ ]:
# Upload the lambda function
# Package up the lambda function code and deploy to Lambda function
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda/lambda_function.py", arcname="lambda_function.py")
z.close()
zip_content = s.getvalue()

lambda_function = lambda_client.update_function_code(
    FunctionName=lambda_function_name,
    ZipFile=zip_content,
    )


### Create IAM policies for agent


In [11]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
try:
    agent_bedrock_policy = iam_client.create_policy(
        PolicyName=agent_bedrock_allow_policy_name,
        PolicyDocument=bedrock_policy_json
    )
except Exception:
    print(f"Policy {agent_bedrock_allow_policy_name} already exists")

Policy RadiologyReportValidator-Agent-ba-us-west-2-575108919340-RadiologyReportValidator already exists


In [12]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
try:
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )


    # Pause to make sure role is created
    time.sleep(10)
        
    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
except Exception as e:
    print(f"Error creating role: {e}. Make sure it doesnt exist already")

Error creating role: An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name AmazonBRkExRoleForAgents_RadiologyReportValidator-Agent already exists.. Make sure it doesnt exist already


### Create Agent

In [22]:
try:
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    agent_id = response['agent']['agentId']
    print(response)

except:
    print("Agent already exists, skipping creation")
    agent_info = [agent for agent in bedrock_agent_client.list_agents()['agentSummaries'] if agent['agentName']==agent_name][0]
    print(agent_info)
    agent_id = agent_info['agentId']
    agent_version = bedrock_agent_client.list_agent_versions(agentId=agent_id)['agentVersionSummaries'][0]['agentVersion']
    response = bedrock_agent_client.list_agent_action_groups(
        agentId=agent_id,
        agentVersion=agent_version
    )
    print(response['actionGroupSummaries'][0])
    action_group_id = response['actionGroupSummaries'][0]['actionGroupId']
    print(f"Using agent_id {agent_id} and action_group_id {action_group_id}")

Agent already exists, skipping creation
{'agentId': 'N8EIVIC9QU', 'agentName': 'RadiologyReportValidator-Agent', 'agentStatus': 'PREPARED', 'description': 'Agent for Validating Radiology Reports', 'updatedAt': datetime.datetime(2025, 4, 6, 22, 12, 5, 490240, tzinfo=tzutc())}
{'actionGroupId': 'FDJLD9TCI2', 'actionGroupName': 'RadiologyActionGroup', 'actionGroupState': 'ENABLED', 'description': 'Actions for Validating Radiology Reports or from Machine Learning Models', 'updatedAt': datetime.datetime(2025, 4, 9, 5, 20, 16, 253607, tzinfo=tzutc())}
Using agent_id N8EIVIC9QU and action_group_id FDJLD9TCI2


## Write Agent functions
Here we create agent functions which we would like to call as the agent processes the input radiology report. A typical agent_function contains the following fields:
- `name`: The name of the function
- `description`: Description of the function
- `parameters`: A dictionary of parameters to be passed in the function 
The `parameter` field contains the name of the parameter, its description, type as well as whether it is a required variable 

In [20]:
agent_functions = [
    {
        'name': 'run_validator',
        'description': 'validates the radiology report for a given report',
        'parameters': {
            "report": {
                "description": "The radiology report for a given patient",
                "required": True,
                "type": "string"
            }
        }
    }, 
    {
        'name' : 'check_radiology_report',
        'description': 'Check of the given text contains a radiology report', 
        'parameters': {
            'report': {
                'description': 'If the text is not a radiology report. Ask for the user to \
                provide the radiology report.',
                'required': True,
                'type': 'string'
            }
        }
    },
    {
        'name': 'download_guidance_document',
        'description': 'Download the ACR guidance document from the S3 bucket. If the document is not \
        available, ask the user to provide the document.',
        'parameters': {
            'report': {
                'description': 'Name of the modality or the anatomical structure to download the ACR guidance document',
                'required': True,
                'type': 'string'
            }
        }
    },
    {
        'name': 'identify_anatomical_structures',
        'description': 'Identify the anatomical structures in the given text. Identify which of the following anatomical structures \
            are present in the text: \
                - Brain \
                - Spine \
                - Chest \
                - Abdomen \
                - Pelvis \
                - Extremities\
                - Transthoracic \
                and return the value from the list.',
        'parameters': {
            'report': {
                'description': 'The radiology report for a given patient',
                'required': True,
                'type': 'string'
            }
        }
    },
    
]

In [23]:
try:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)
except:
    print("Action group already exists")
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        actionGroupId=action_group_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function_arn
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )
    print(agent_action_group_response)

Action group already exists
{'ResponseMetadata': {'RequestId': 'c4a36656-1c6b-4d44-acea-6eaeacaa9624', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 12 Apr 2025 07:47:57 GMT', 'content-type': 'application/json', 'content-length': '2082', 'connection': 'keep-alive', 'x-amzn-requestid': 'c4a36656-1c6b-4d44-acea-6eaeacaa9624', 'x-amz-apigw-id': 'I5kbFGinPHcEYlA=', 'x-amzn-trace-id': 'Root=1-67fa1aac-2450d45627159d7c3742ec94'}, 'RetryAttempts': 0}, 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:575108919340:function:RadiologyReportValidator-575108919340-Lambda'}, 'actionGroupId': 'FDJLD9TCI2', 'actionGroupName': 'RadiologyActionGroup', 'actionGroupState': 'ENABLED', 'agentId': 'N8EIVIC9QU', 'agentVersion': 'DRAFT', 'clientToken': 'a4a6af36-1a1a-4445-9e89-3386770e5115', 'createdAt': datetime.datetime(2025, 4, 6, 22, 17, 25, 333082, tzinfo=tzutc()), 'description': 'Actions for Validating Radiology Reports or from Machine Learning Models', 'functionSch

### Permissions for the lambda function
In this lamda function, required permissions are given. Here we are using bedrock in the agent, so appropriate permissions are given.

In [24]:
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId='allow_bedrock',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except: 
    print("Permission already exists")

Permission already exists


#### Prepare the agent

In [25]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': 'faf27089-a0e7-49ce-888d-a4a798e21685', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 12 Apr 2025 07:48:07 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'faf27089-a0e7-49ce-888d-a4a798e21685', 'x-amz-apigw-id': 'I5kcoE1yvHcEaHQ=', 'x-amzn-trace-id': 'Root=1-67fa1ab6-6370c48e203463dc01b7e527'}, 'RetryAttempts': 0}, 'agentId': 'N8EIVIC9QU', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 4, 12, 7, 48, 7, 137643, tzinfo=tzutc())}


## Test case

In [27]:

test_query ="Here is the radiology report: FINAL REPORT EXAMINATION : CHEST ( PORTABLE AP ) INDICATION : History :___ F with ett placement TECHNIQUE : Upright AP view of the chest COMPARISON : None . Patient is currently listed as EU critical . \
FINDINGS : Endotracheal tube tip terminates approximately a 4.6 cm from the carina . Enteric tube tip terminates within the distal esophagus and should be advanced by at least 11 cm .Heart size is normal . Gaseous distension of the stomach is noted "

session_id:str = str(uuid.uuid1())
print(agent_id)

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer=""
for event in response["completion"]:
    print(event.keys())
    print(event)
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

N8EIVIC9QU
dict_keys(['trace'])
{'trace': {'agentAliasId': 'TSTALIASID', 'agentId': 'N8EIVIC9QU', 'agentVersion': 'DRAFT', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-west-2:575108919340:agent-alias/N8EIVIC9QU/TSTALIASID'}], 'eventTime': datetime.datetime(2025, 4, 12, 7, 48, 40, 627205, tzinfo=tzutc()), 'sessionId': '8ca5aeee-1772-11f0-83af-f204dc4090c7', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</function_calls>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are a Radiology Report Validator, helping junior radiologist write reports in adherence to the ACR guidance criterion. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>

In [28]:
# Print the final answer
print(answer)

The provided chest radiograph report does not fully adhere to the ACR guidelines. It is missing key components like:

- Patient positioning and degree of inspiration
- Systematic evaluation of lung fields, bony structures, mediastinum, and pleural spaces  
- Mention of any consolidations, effusions, or masses
- An overall impression summarizing the significant findings

To improve the report and meet the ACR standards, the radiologist should include these missing elements in a more comprehensive and structured format.
